In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashplayerstats

data_DIR = "../data/box/"
fig_DIR = "../figs/player_analysis/"

In [ ]:
dfa = []
for iu in range(6):
    url = f"https://www.teamrankings.com/nba/player-stat/fouls-technical?rate=season-totals&season_id={216+iu}"
    df1 = pd.read_html(url)
    df2 = df1[0]
    df2["season"] = 2019 + iu
    dfa.append(df2)
df = pd.concat(dfa)

In [ ]:
dft = df.groupby("Player")["Value"].sum().reset_index().sort_values("Value",ascending=False).reset_index(drop=True)
dft = dft.rename(columns={"Value":"Fouls"})
dft = dft.nlargest(10,"Fouls")
dft = dft.reset_index()
dft["index"] = dft["index"] + 1

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%%R -i dft
df <- dft
library(tidyverse)
library(gt)
library(gtExtras)
df %>% 
  gt()%>%
  tab_header(
    title = md("**Most Technical Fouls Since 2018-19 NBA Season**"),
    ) %>%
    cols_label( index="#"
    ) %>%
    gt_plt_bar(column = Fouls, keep_column = TRUE, color = "red") %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 16,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: teamrankings.com/nba" ) %>% gtsave("../figs/player_analysis/technical_fouls.png",size=5) 